In [1]:
import pysam
import pandas as pd
import json
import re
from typing import List, Any

In [7]:
%cd /mnt/c/users/Parv/doc/Helixworks/Basecalling/
%ls

/mnt/c/users/Parv/doc/Helixworks/Basecalling
 Basecalling/
 DeepSimulator/
'Scaling Logical Density using Motif Based Approach.pdf'*
 calls_summary.tsv*
 code/
 cycle_dataset/
 deepNano.pdf*
 dorado-0.5.1-win64/
 env/
 figures/
'improving nanopore.pdf'*
 motif_search_limits.png*
 presentations/
 reads_bonito/
 report-17.pdf*
 squigulator/


In [9]:
%cd squigulator/slow5tools-v1.1.0/

/mnt/c/users/Parv/doc/Helixworks/Basecalling/squigulator/slow5tools-v1.1.0


In [12]:
%mv testing.fast5 /mnt/c/users/Parv/doc/Helixworks/Basecalling/dorado-0.5.1-win64/

In [27]:
%cd /mnt/c/users/Parv/doc/Helixworks/Basecalling/dorado-0.5.1-win64/dorado-0.5.1-win64/bin

/mnt/c/users/Parv/doc/Helixworks/Basecalling/dorado-0.5.1-win64/dorado-0.5.1-win64/bin


In [46]:
! ./dorado.exe basecaller dna_r9.4.1_e8_hac@v3.3 testing.fast5 > calls.bam

[2025-03-12 12:35:30.002] [info] > Creating basecall pipeline
[2025-03-12 12:35:33.342] [info]  - set batch size for cuda:0 to 512
[2025-03-12 12:35:33.366] [warning] FAST5 support is unoptimized and will result in poor performance. Please convert your dataset to POD5: https://pod5-file-format.readthedocs.io/en/latest/docs/tools.html#pod5-convert-fast5
[2025-03-12 12:36:10.732] [info] > Simplex reads basecalled: 20000
[2025-03-12 12:36:10.732] [info] > Basecalled @ Samples/s: 3.530267e+06
[2025-03-12 12:36:10.770] [info] > Finished


In [47]:
%pwd

'/mnt/c/users/Parv/doc/Helixworks/Basecalling/dorado-0.5.1-win64/dorado-0.5.1-win64/bin'

In [52]:

## original bases and motifs (pre sequencing)

with open(r"mnt/c/users/Parv/doc/HelixWorks/Basecalling/code/motifcaller/notebooks/info.json", 'r') as f:
    info_dict = json.load(f)

read_ids = info_dict['read_ids']

FileNotFoundError: [Errno 2] No such file or directory: 'mnt/c/users/Parv/doc/HelixWorks/Basecalling/code/motifcaller/notebooks/info.json'

In [33]:
motif_picks = info_dict['motif_choices']
motif_labels = info_dict['motif_labels']
motif_links = info_dict['link_choices']

In [68]:
motif_picks

['CCACTAACGTAGAGTACTGC',
 'GTCGAGGATAATTGTGCAGA',
 'CACCCCCGTATTTTGAGCGG',
 'ATCCTAGCGTTTTCGAACGT',
 'CACGTTGTTTTATCGTCGTC',
 'GTCTCCCCAGGAACGGTGCC',
 'GCATGAAGACACTAGCGCCT',
 'CAGCGCCACCAACTCCACAA',
 'GTGCGGCCGT',
 'GGGACAGCTA',
 'ACATCAGTCG',
 'CCTTCATACC',
 'GTCCATCCAC',
 'ATGACGTCGG',
 'AGCTTTGAGC',
 'TTCACACCTT']

In [53]:


def get_read_id(split_x: List[Any]) -> str:
    x_id = split_x[0]
    pattern = r">.*!"
    return re.search(r">.*!", x_id)[0][1:-10].replace('!', '')

def get_base_sequence(split_x: List[Any]) -> str:
    return split_x[9].strip()

def create_base_seq_from_motifs(
        motif_seq: List[List[int]], motif_picks:List[str],
        motif_links: List[str]) -> str:
    
    return "".join([
        motif_picks[i-1] for i in motif_seq])



def motif_search(base_seq: str, motif_picks: List[str], motif_links: List[str]):

    motif_length = len(motif_picks[0]) - 13
    motif_prediction = []
    # Sliding window searching for motif picks
    for i in range(0, len(base_seq) - motif_length, motif_length):
        subseq = base_seq[i: i + motif_length]
        for ind, j in enumerate(motif_picks):
            if subseq in j:
                motif_prediction.append(ind+1)

    return motif_prediction


In [58]:

def get_base_predictions_df(bam_filepath: str) -> pd.DataFrame:

    samfile = pysam.AlignmentFile(
        bam_filepath, "rb", check_sq=False)
    seq_id = []
    positions = []

    read_ids_fasta = []
    read_ids_fast5 = []
    base_seq_predictions = []

    iter = samfile.fetch(until_eof=True)
    for ind, x in enumerate(iter):
        x = str(x)
        split_x = x.split()
        strand_orientation = split_x[0][-1]
        read_id_fasta = get_read_id(split_x)
        base_seq_prediction = get_base_sequence(split_x)

        base_seq_predictions.append(base_seq_prediction)
        read_ids_fasta.append(read_id_fasta)
        read_ids_fast5.append(split_x[0])

    df = pd.DataFrame()
    df['read_ids_fasta'] = read_ids_fasta
    df['read_ids_fast5'] = read_ids_fast5
    df['base_predictions'] = base_seq_predictions

    return df
        

In [59]:
df = get_base_predictions_df("/mnt/c/users/Parv/doc/Helixworks/Basecalling/dorado-0.5.1-win64/dorado-0.5.1-win64/bin/calls.bam")

In [67]:
df.to_csv("basecalled.csv")

In [66]:
%cd /mnt/c/users/Parv/doc/Helixworks/Basecalling/code/motifcaller/

/mnt/c/users/Parv/doc/Helixworks/Basecalling/code/motifcaller


In [55]:

# Squiggle database (that needs added motif search predictions)
dataset_path = r"/mnt/c/Users/Parv/Doc/HelixWorks/Basecalling/code/motifcaller/data/synthetic/pickled_datasets/25_2_25.pkl"
dataset = pd.read_pickle(dataset_path)


ModuleNotFoundError: No module named 'numpy._core.numeric'